# DATA CLEANING

Here I am cleaning the two London datasets scraped from Zoopla:
- The first contains current listings on Zoopla with asking price
- The second contains ~350,000 historical listings with last sold price.

n.b. The 350k will be extended to contain all available 3.5 million listings in a future scrape

# GENERAL IMPORTS

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import time
from math import ceil

# CURRENT LISTINGS DATASET

In [4]:
current_data ="data/csv_data/london_scrape.csv"

In [5]:
df2 = pd.read_csv(current_data, sep='\t')

In [6]:
df2.head()

,address,price,bedrooms,bathrooms,lounges,url,nearby_station_1,nearby_station_2
0,"Park Street, Chelsea/London SW6","£1,375,000",2,2,1,/new-homes/details/61623573/?search_identifier...,0.1 miles Imperial Wharf,0.2 miles Chelsea Harbour Pier
1,"Bridges Court, London SW11","£440,000",1,1,1,/for-sale/details/61567435/?search_identifier=...,0.4 miles Chelsea Harbour Pier,0.4 miles Imperial Wharf
2,"Carshalton Grove, Sutton SM1","Guide price£650,000",3,2,2,/for-sale/details/61623414/?search_identifier=...,0.6 miles Carshalton,0.6 miles Sutton (London)
3,"Rutland Rd, Forest Gate E7","Guide price£545,000",3,1,2,/for-sale/details/61623411/?search_identifier=...,0.4 miles East Ham,0.5 miles Upton Park
4,"Rutland Rd, Forest Gate E7","Guide price£545,000",3,1,2,/for-sale/details/61623405/?search_identifier=...,0.4 miles East Ham,0.5 miles Upton Park
...,...,...,...,...,...,...,...,...
9995,"Burlington Road, Osterley, Isleworth TW7","£750,000",3,1,2,/for-sale/details/61476741/?search_identifier=...,0.3 miles Osterley,0.5 miles Hounslow East
9996,"Tufnell Park, Tufnell Park, London N7","£375,000",1,1,na,/for-sale/details/61476757/?search_identifier=...,0.5 miles Upper Holloway,0.5 miles Tufnell Park
9997,"Holmewood Gardens, Brixton Hill, London SW2","£700,000",2,1,na,/for-sale/details/61476758/?search_identifier=...,0.6 miles Streatham Hill,0.8 miles Tulse Hill
9998,"Coldershaw Road, Ealing W13","Guide price£535,000",2,1,1,/for-sale/details/61476750/?search_identifier=...,0.5 miles West Ealing,0.6 miles Drayton Green


## Nulls

In [7]:
df2.replace(re.compile('^na$'), np.NaN, inplace=True)

In [8]:
df2.price

0                £1,375,000
1                  £440,000
2       Guide price£650,000
3       Guide price£545,000
4       Guide price£545,000
               ...         
9995               £750,000
9996               £375,000
9997               £700,000
9998    Guide price£535,000
9999               £950,000
Name: price, Length: 10000, dtype: object

## Cleaning Price Data

In [9]:
df2['price_gbp'] = df2[['price']].replace(re.compile(r'[£,\D]'), '').replace(re.compile(r'^$'), np.NaN).astype('float') #.apply(lambda x: np.NaN if x == '' else x) #.astype('float')   #replace(re.compile(r'^$'), None) #.astype('float')

In [10]:
df2.price_gbp #.replace(re.compile(r'^$'), np.NaN).astype('float')

0       1375000.0
1        440000.0
2        650000.0
3        545000.0
4        545000.0
          ...    
9995     750000.0
9996     375000.0
9997     700000.0
9998     535000.0
9999     950000.0
Name: price_gbp, Length: 10000, dtype: float64

## Bedrooms, Bathrooms & Lounges 'na' to np.NaN

In [11]:
for col in ['bedrooms', 'bathrooms', 'lounges']:
    df2[col] = df2[col].replace(np.NaN, None).astype('int')

In [12]:
df2.drop(columns=['price'], inplace= True)

In [13]:
(~df2.nearby_station_1.isnull()).sum()

9856

In [14]:
df2.nearby_station_1.str.contains('miles').sum()

9856

## Cleaning Nearby Places of Interest

In [15]:
df2[['distance_to_station_1_miles', 'nearby_station_1']] = df2.nearby_station_1.str.split('miles', expand=True)

In [16]:
df2[['distance_to_station_2_miles', 'nearby_station_2']] = df2.nearby_station_2.str.split('miles', expand=True)

In [17]:
df2.address.str.split( r"([Gg][Ii][Rr] 0[Aa]{2})|((([A-Za-z][0-9]{1,2})|(([A-Za-z][A-Ha-hJ-Yj-y][0-9]{1,2})|(([A-Za-z][0-9][A-Za-z])|([A-Za-z][A-Ha-hJ-Yj-y][0-9][A-Za-z]?))))\s?[0-9][A-Za-z]{2})", expand=True)

,0
0,"Park Street, Chelsea/London SW6"
1,"Bridges Court, London SW11"
2,"Carshalton Grove, Sutton SM1"
3,"Rutland Rd, Forest Gate E7"
4,"Rutland Rd, Forest Gate E7"
...,...
9995,"Burlington Road, Osterley, Isleworth TW7"
9996,"Tufnell Park, Tufnell Park, London N7"
9997,"Holmewood Gardens, Brixton Hill, London SW2"
9998,"Coldershaw Road, Ealing W13"


In [18]:
df2[['address_no_code', 'postcode_first_half']] = df2.address.str.split(r'([A-Z]{1,2}[0-9]{1,2})', expand=True).iloc[:,[0,1]]

In [19]:
df2.columns

Index(['address', 'bedrooms', 'bathrooms', 'lounges', 'url',
       'nearby_station_1', 'nearby_station_2', 'price_gbp',
       'distance_to_station_1_miles', 'distance_to_station_2_miles',
       'address_no_code', 'postcode_first_half'],
      dtype='object')

In [20]:
reorder = ['address', 'address_no_code' , 'postcode_first_half', 'bedrooms', 'bathrooms', 'lounges', 'url',
       'nearby_station_1', 'distance_to_station_1_miles', 'nearby_station_2', 'distance_to_station_2_miles', 'price_gbp']

In [21]:
df2 = df2[reorder]

In [22]:
!ls data/csv_data_clean/


london_current.csv


In [23]:
path = 'data/csv_data_clean/london_current.csv'
#df2.to_csv(path, index=False)

In [24]:
pd.read_csv(path)

,address,address_no_code,postcode_first_half,bedrooms,bathrooms,lounges,url,nearby_station_1,distance_to_station_1_miles,nearby_station_2,distance_to_station_2_miles,price_gbp
0,"Park Street, Chelsea/London SW6","Park Street, Chelsea/London",SW6,2,2,1,/new-homes/details/61623573/?search_identifier...,Imperial Wharf,0.1,Chelsea Harbour Pier,0.2,1375000.0
1,"Bridges Court, London SW11","Bridges Court, London",SW11,1,1,1,/for-sale/details/61567435/?search_identifier=...,Chelsea Harbour Pier,0.4,Imperial Wharf,0.4,440000.0
2,"Carshalton Grove, Sutton SM1","Carshalton Grove, Sutton",SM1,3,2,2,/for-sale/details/61623414/?search_identifier=...,Carshalton,0.6,Sutton (London),0.6,650000.0
3,"Rutland Rd, Forest Gate E7","Rutland Rd, Forest Gate",E7,3,1,2,/for-sale/details/61623411/?search_identifier=...,East Ham,0.4,Upton Park,0.5,545000.0
4,"Rutland Rd, Forest Gate E7","Rutland Rd, Forest Gate",E7,3,1,2,/for-sale/details/61623405/?search_identifier=...,East Ham,0.4,Upton Park,0.5,545000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,"Burlington Road, Osterley, Isleworth TW7","Burlington Road, Osterley, Isleworth",TW7,3,1,2,/for-sale/details/61476741/?search_identifier=...,Osterley,0.3,Hounslow East,0.5,750000.0
9996,"Tufnell Park, Tufnell Park, London N7","Tufnell Park, Tufnell Park, London",N7,1,1,2,/for-sale/details/61476757/?search_identifier=...,Upper Holloway,0.5,Tufnell Park,0.5,375000.0
9997,"Holmewood Gardens, Brixton Hill, London SW2","Holmewood Gardens, Brixton Hill, London",SW2,2,1,2,/for-sale/details/61476758/?search_identifier=...,Streatham Hill,0.6,Tulse Hill,0.8,700000.0
9998,"Coldershaw Road, Ealing W13","Coldershaw Road, Ealing",W13,2,1,1,/for-sale/details/61476750/?search_identifier=...,West Ealing,0.5,Drayton Green,0.6,535000.0


In [25]:
df2.iloc[:,1]

0                   Park Street, Chelsea/London 
1                         Bridges Court, London 
2                      Carshalton Grove, Sutton 
3                       Rutland Rd, Forest Gate 
4                       Rutland Rd, Forest Gate 
                          ...                   
9995       Burlington Road, Osterley, Isleworth 
9996         Tufnell Park, Tufnell Park, London 
9997    Holmewood Gardens, Brixton Hill, London 
9998                    Coldershaw Road, Ealing 
9999                   Kilravock Street, London 
Name: address_no_code, Length: 10000, dtype: object

# HISTORICAL PRICE DATASET

In [1]:
data ="data/csv_data/london_historical_scrape.csv"

In [4]:
df =pd.read_csv(data, sep='\t')

## Splitting Address To Get Postcode

In [6]:
df[df.address.str.contains(r"24 Oaklands Avenue, Sidcup, DA15 8NB")]

,address,last_sold_date,last_sold_price,estimated_price,bedrooms,bathrooms,lounges,url,tenure,property_type
308203,"24 Oaklands Avenue, Sidcup, DA15 8NB",Last sold,-,"£435,000 - £532,000",NaN,NaN,NaN,/property/uprn/100020257388/,freehold,semi-detached house


In [5]:
df

,address,last_sold_date,last_sold_price,estimated_price,bedrooms,bathrooms,lounges,url,tenure,property_type
0,"1 Bickley Crescent, Bromley, BR1 2DN",Last sold - Oct 1999,"£97,500","£461,000 - £564,000",2.0,NaN,2.0,/property/uprn/100020390465/,freehold,semi-detached house
1,"1 Chadd Drive, Bromley, BR1 2DP",Last sold - Jul 2018,"£675,000","£789,000 - £872,000",NaN,NaN,NaN,/property/uprn/100020393770/,freehold,detached house
2,"1 Page Heath Lane, Bromley, BR1 2DR",Last sold - Jan 2007,"£480,000","£740,000 - £905,000",4.0,2.0,2.0,/property/uprn/100020408794/,freehold,semi-detached house
3,"1 Beresford Drive, Bromley, BR1 2DU",Last sold - Nov 2006,"£420,000","£830,000 - £918,000",3.0,1.0,3.0,/property/uprn/100020390386/,freehold,detached house
4,"1 St Michael's Close, Bromley, BR1 2DX",Last sold - Nov 2006,"£376,000","£740,000 - £904,000",NaN,NaN,NaN,/property/uprn/100020416127/,freehold,detached house
...,...,...,...,...,...,...,...,...,...,...
308341,"25 Orchard Rise East, Sidcup, DA15 8RU",Last sold - Aug 2005,"£180,000","£411,000 - £454,000",3.0,NaN,2.0,/property/uprn/100020257952/,freehold,mid terrace house
308342,"25 Chester Road, Sidcup, DA15 8RX",Last sold - May 2018,"£323,000","£386,000 - £426,000",3.0,1.0,1.0,/property/uprn/100020247967/,freehold,terrace property
308343,"25 Ridgeway East, Sidcup, DA15 8RY",Last sold,-,"£402,000 - £445,000",NaN,NaN,NaN,/property/uprn/100020259734/,freehold,terrace property
308344,"25 Ridgeway West, Sidcup, DA15 8SF",Last sold,-,"£394,000 - £482,000",NaN,NaN,NaN,/property/uprn/100020259766/,freehold,terrace property


In [109]:
df[['unprocessed_address', 'postcode']] = df.address.str.split( r"([Gg][Ii][Rr] 0[Aa]{2})|((([A-Za-z][0-9]{1,2})|(([A-Za-z][A-Ha-hJ-Yj-y][0-9]{1,2})|(([A-Za-z][0-9][A-Za-z])|([A-Za-z][A-Ha-hJ-Yj-y][0-9][A-Za-z]?))))\s?[0-9][A-Za-z]{2})", expand=True).loc[:,[0,2]]

In [110]:
df[['paon', 'address_processing_2']] = df.unprocessed_address.str.split(",", n=1, expand=True).iloc[:]

In [112]:
#df.address_processing_2.str.split("[0-9]+[],", expand=True, n=1).loc[:,1].value_counts()

In [113]:
df.address_processing_2.str.split("([a-zA-Z -,]*[0-9]+[a-zA-Z -]*,)", expand=True).loc[:,1].value_counts()

 Pinnacle Apartments, 11 Saffron Central Square,    240
 Tennyson Apartments, 1 Saffron Central Square,     207
 Bertha James Court, 32 Masons Hill,                204
 1 Newgate,                                         183
 Rosing Apartments, 45 Homesdale Road,              164
                                                   ... 
 13 Bywood Avenue,                                    1
 13 Farquharson Road,                                 1
 13 Duppas Hill Road,                                 1
 13 Witherby Close,                                   1
 16 Christchurch Road,                                1
Name: 1, Length: 4918, dtype: int64

## Cleaning and Splitting Scraped Last Sold Date (Month-Year)

In [114]:
df[['month_sold_last', 'year_sold_last']] = df.last_sold_date.str.split("Last sold[ -]*([A-Za-z]{3}) ([0-9]{4})", expand=True).loc[:,[1,2]]

## Cleaning Price Data (Last Sold & Zoopla Estimates)

In [115]:
df["last_sold_price_gbp"] = df.last_sold_price.str.replace("£","").str.replace(",","").replace("-", "").replace(re.compile(r"^$"), np.NaN).astype("float")

In [116]:
df[['min_estimate_gbp','max_estimate_gbp']] = df.estimated_price.str.split(r"£([0-9,.]+) - £([0-9,.]+)", expand=True).iloc[:,[1,2]]

In [117]:
df.min_estimate_gbp = df.min_estimate_gbp.str.replace("£","").str.replace(",","").astype("float")

In [118]:
df.max_estimate_gbp = df.max_estimate_gbp.str.replace("£","").str.replace(",","").astype("float")

## Cleaning Scraped Bedroom, Bathroom & Lounge Counts

In [119]:
for col in ['bedrooms', 'bathrooms', 'lounges']:
    df[col] = df[col].replace(np.NaN, None) #.astype('int')

In [120]:
df.bathrooms.replace(np.NaN, None)

0         NaN
1         NaN
2         2.0
3         1.0
4         1.0
         ... 
308341    1.0
308342    1.0
308343    1.0
308344    1.0
308345    1.0
Name: bathrooms, Length: 308346, dtype: float64

In [121]:
df['lounges'] = df.lounges.replace(np.NaN, None).astype('int')

In [122]:
df.drop(columns=['last_sold_date','last_sold_price','estimated_price'], inplace=True )

In [123]:
col_reorder = ['address', 'unprocessed_address',  'paon',
       'saon', 'postcode', 'bedrooms', 'bathrooms', 'lounges', 'url', 'tenure',
       'property_type', 'month_sold_last', 'year_sold_last',
       'last_sold_price_gbp', 'min_estimate_gbp', 'max_estimate_gbp']

In [124]:
df.rename(columns={'address_processing_2':'saon'}, inplace=True)

In [125]:
df = df[col_reorder]

In [127]:
df['saon'] = df.saon.str.replace(re.compile(r", *$"), "")

In [129]:
outfile = 'data/csv_data_clean/london_historical.csv'

In [132]:
df.to_csv(outfile, index=False)

In [133]:
pd.read_csv(outfile)

,address,unprocessed_address,paon,saon,postcode,bedrooms,bathrooms,lounges,url,tenure,property_type,month_sold_last,year_sold_last,last_sold_price_gbp,min_estimate_gbp,max_estimate_gbp
0,"1 Bickley Crescent, Bromley, BR1 2DN","1 Bickley Crescent, Bromley,",1 Bickley Crescent,Bromley,BR1 2DN,2.0,NaN,2,/property/uprn/100020390465/,freehold,semi-detached house,Oct,1999.0,97500.0,461000.0,564000.0
1,"1 Chadd Drive, Bromley, BR1 2DP","1 Chadd Drive, Bromley,",1 Chadd Drive,Bromley,BR1 2DP,2.0,NaN,2,/property/uprn/100020393770/,freehold,detached house,Jul,2018.0,675000.0,789000.0,872000.0
2,"1 Page Heath Lane, Bromley, BR1 2DR","1 Page Heath Lane, Bromley,",1 Page Heath Lane,Bromley,BR1 2DR,4.0,2.0,2,/property/uprn/100020408794/,freehold,semi-detached house,Jan,2007.0,480000.0,740000.0,905000.0
3,"1 Beresford Drive, Bromley, BR1 2DU","1 Beresford Drive, Bromley,",1 Beresford Drive,Bromley,BR1 2DU,3.0,1.0,3,/property/uprn/100020390386/,freehold,detached house,Nov,2006.0,420000.0,830000.0,918000.0
4,"1 St Michael's Close, Bromley, BR1 2DX","1 St Michael's Close, Bromley,",1 St Michael's Close,Bromley,BR1 2DX,3.0,1.0,3,/property/uprn/100020416127/,freehold,detached house,Nov,2006.0,376000.0,740000.0,904000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308341,"25 Orchard Rise East, Sidcup, DA15 8RU","25 Orchard Rise East, Sidcup,",25 Orchard Rise East,Sidcup,DA15 8RU,3.0,1.0,2,/property/uprn/100020257952/,freehold,mid terrace house,Aug,2005.0,180000.0,411000.0,454000.0
308342,"25 Chester Road, Sidcup, DA15 8RX","25 Chester Road, Sidcup,",25 Chester Road,Sidcup,DA15 8RX,3.0,1.0,1,/property/uprn/100020247967/,freehold,terrace property,May,2018.0,323000.0,386000.0,426000.0
308343,"25 Ridgeway East, Sidcup, DA15 8RY","25 Ridgeway East, Sidcup,",25 Ridgeway East,Sidcup,DA15 8RY,3.0,1.0,1,/property/uprn/100020259734/,freehold,terrace property,NaN,NaN,NaN,402000.0,445000.0
308344,"25 Ridgeway West, Sidcup, DA15 8SF","25 Ridgeway West, Sidcup,",25 Ridgeway West,Sidcup,DA15 8SF,3.0,1.0,1,/property/uprn/100020259766/,freehold,terrace property,NaN,NaN,NaN,394000.0,482000.0


# CLEANING (FROM EDA NOTEBOOK)

## IMPORT DATASETS

In [2]:
hist_data = 'data/csv_data_clean/london_historical.csv' 
curr_data = 'data/csv_data_clean/london_current.csv' 

In [3]:
hist_df = pd.read_csv(hist_data)

In [4]:
hist_df

,address,unprocessed_address,paon,saon,postcode,bedrooms,bathrooms,lounges,url,tenure,property_type,month_sold_last,year_sold_last,last_sold_price_gbp,min_estimate_gbp,max_estimate_gbp
0,"1 Bickley Crescent, Bromley, BR1 2DN","1 Bickley Crescent, Bromley,",1 Bickley Crescent,Bromley,BR1 2DN,2.0,NaN,2,/property/uprn/100020390465/,freehold,semi-detached house,Oct,1999.0,97500.0,461000.0,564000.0
1,"1 Chadd Drive, Bromley, BR1 2DP","1 Chadd Drive, Bromley,",1 Chadd Drive,Bromley,BR1 2DP,2.0,NaN,2,/property/uprn/100020393770/,freehold,detached house,Jul,2018.0,675000.0,789000.0,872000.0
2,"1 Page Heath Lane, Bromley, BR1 2DR","1 Page Heath Lane, Bromley,",1 Page Heath Lane,Bromley,BR1 2DR,4.0,2.0,2,/property/uprn/100020408794/,freehold,semi-detached house,Jan,2007.0,480000.0,740000.0,905000.0
3,"1 Beresford Drive, Bromley, BR1 2DU","1 Beresford Drive, Bromley,",1 Beresford Drive,Bromley,BR1 2DU,3.0,1.0,3,/property/uprn/100020390386/,freehold,detached house,Nov,2006.0,420000.0,830000.0,918000.0
4,"1 St Michael's Close, Bromley, BR1 2DX","1 St Michael's Close, Bromley,",1 St Michael's Close,Bromley,BR1 2DX,3.0,1.0,3,/property/uprn/100020416127/,freehold,detached house,Nov,2006.0,376000.0,740000.0,904000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308341,"25 Orchard Rise East, Sidcup, DA15 8RU","25 Orchard Rise East, Sidcup,",25 Orchard Rise East,Sidcup,DA15 8RU,3.0,1.0,2,/property/uprn/100020257952/,freehold,mid terrace house,Aug,2005.0,180000.0,411000.0,454000.0
308342,"25 Chester Road, Sidcup, DA15 8RX","25 Chester Road, Sidcup,",25 Chester Road,Sidcup,DA15 8RX,3.0,1.0,1,/property/uprn/100020247967/,freehold,terrace property,May,2018.0,323000.0,386000.0,426000.0
308343,"25 Ridgeway East, Sidcup, DA15 8RY","25 Ridgeway East, Sidcup,",25 Ridgeway East,Sidcup,DA15 8RY,3.0,1.0,1,/property/uprn/100020259734/,freehold,terrace property,NaN,NaN,NaN,402000.0,445000.0
308344,"25 Ridgeway West, Sidcup, DA15 8SF","25 Ridgeway West, Sidcup,",25 Ridgeway West,Sidcup,DA15 8SF,3.0,1.0,1,/property/uprn/100020259766/,freehold,terrace property,NaN,NaN,NaN,394000.0,482000.0
